<a href="https://colab.research.google.com/github/ByeongHyunPak/omni-proj/blob/main/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorboardX

In [ ]:
!git clone https://github.com/ByeongHyunPak/omni-proj.git

In [ ]:
import os
os.chdir('/content/omni-proj')
!git pull

In [ ]:
import os
os.chdir('/content/omni-proj/omni-proj')

In [ ]:
from PIL import Image
import os

def load_images_from_folder(folder):
  images = []
  for filename in os.listdir(folder):
    img_path = os.path.join(folder, filename)
    if os.path.isfile(img_path) and img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
      try:
        img = Image.open(img_path)
        images.append(img)
      except Exception as e:
        print(f"Error loading image {img_path}: {e}")
  return images

imgs_folder = '/content/omni-proj/imgs/erps' 
images = load_images_from_folder(imgs_folder)

for img in images:
    img = pil_to_tensor(img)
    print(img.shape)

In [ ]:
import utils
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

def pil_to_tensor(pil_image):
  transform = transforms.ToTensor()
  tensor_image = transform(pil_image)
  return tensor_image

def tensor_to_pil(tensor_image):
    transform = transforms.ToPILImage()
    pil_image = transform(tensor_image)
    return pil_image

def erp2per(hr_erp_img, THE, PHI):
    hr_erp_img = pil_to_tensor(hr_erp_img)

    H, W = hr_erp_img.shape[-2:]
    HWy = (H//2, H//2)
    HWx = (H, W)

    FOVy = 90
    FOVx = 360

    gridy = utils.make_coord(HWy)
    gridy2x, masky = utils.gridy2x_erp2per(
        gridy, HWy, HWx, THE, PHI, FOVy, FOVx)
    gridy2x = gridy2x.view(*HWy, 2)

    inp = F.grid_sample(hr_erp_img.unsqueeze(0),
                        gridy2x.unsqueeze(0).flip(-1),
                        mode='bicubic',
                        align_corners=False).clamp_(0, 1)[0]

    gridx = utils.make_coord(HWx, flatten=False)
    gridx2y, maskx = utils.gridy2x_per2erp(
        gridx, HWx, HWy, THE, PHI, FOVx, FOVy)
    
    maskx = maskx.view(1, *HWx)
    valid_hr_erp_img = hr_erp_img * maskx
    
    return inp, valid_hr_erp_img

In [ ]:
import random

THE = random.uniform(-135, 135)
PHI = random.uniform(-45, 45)

hr_erp_img = images[1]
pers_img, valid_hr_erp_img = erp2per(hr_erp_img, THE, PHI)
pers_img = tensor_to_pil(pers_img)
valid_hr_erp_img = tensor_to_pil(valid_hr_erp_img)

display(hr_erp_img)
display(valid_hr_erp_img)
display(pers_img)


In [ ]:
pers_img_imgs_folder = '/content/omni-proj/imgs/pers' 
pers_images = load_images_from_folder(pers_img_imgs_folder)

for img in pers_images:
    img = pil_to_tensor(img)
    print(img.shape)

In [ ]:
def erp2per(hr_per_img, THE, PHI):
    hr_per_img = pil_to_tensor(hr_per_img)

    H, W = hr_per_img.shape[-2:]
    HWy = (H, 2*H)
    HWx = (H, W)

    FOVy = 360
    FOVx = 90

    gridy = utils.make_coord(HWy)
    gridy2x, masky = utils.gridy2x_per2erp(
        gridy, HWy, HWx, THE, PHI, FOVy, FOVx)
    gridy2x = gridy2x.view(*HWy, 2)
    masky = masky.view(1, *HWy)

    inp = F.grid_sample(hr_per_img.unsqueeze(0),
                        gridy2x.unsqueeze(0).flip(-1),
                        mode='bicubic',
                        align_corners=False).clamp_(0, 1)[0]
    inp = inp * masky
    
    gridx = utils.make_coord(HWx, flatten=False)
    gridx2y, maskx = utils.gridy2x_per2erp(
        gridx, HWx, HWy, THE, PHI, FOVx, FOVy)    
    return inp

In [ ]:
import random

THE = random.uniform(-135, 135)
PHI = random.uniform(-45, 45)

hr_per_img = pers_images[0]
pers_img = erp2per(hr_per_img, THE, PHI)
pers_img = tensor_to_pil(pers_img)

display(hr_per_img)
display(pers_img)